In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv,global_max_pool as gmp


# GCN based model
class GCNNet(torch.nn.Module):
    def __init__(self,n_output=1,n_filters=32,embed_dim=128,num_features_xd=78,num_features_xt=25,output_dim=128,dropout=0.2):
        '''
        定义一个名为GCNNet的神经网络模型，它继承自torch.nn.Module类。
        在__init__函数中定义了模型的各种参数，包括输出层的数量（n_output）、过滤器数量（n_filters）、输入特征的维度（embed_dim）、药物特征的数量（num_features_xd）、蛋白质特征的数量（num_features_xt）、输出特征的维度（output_dim）和dropout率（dropout）等。
        '''
        super(GCNNet,self).__init__()   #这段代码是调用父类的初始化函数，以便在新的类GCNNet中继承所有父类的属性和方法，方便调用

        #处理SMILES的神经网络模型
        self.n_output = n_output                                 #表示输出的维度即模型最后一层的输出维度
        self.conv1 = GCNConv(num_features_xd,num_features_xd)    #这行代码创建了一个实例，并将其赋值给属性变量self.conv1，该层是基于图形数据的卷积神经网络，适用于图形数据的特征提取，两个参数分别代表输入输出维度，在forward函数中通过调用self.conv1(x,edge_index)来进行图卷积操作，这个也就是第一层的图卷积神经网络
        self.conv2 = GCNConv(num_features_xd,num_features_xd*2)  #创建第二层图卷积神经网络，同上只不过这里输入输出维度变了注意一下
        self.conv3 = GCNConv(num_features_xd*2,num_features_xd*4)#创建第三层图卷积神经网络，这里输入输出维度也改变了
        self.fc_g1 = torch.nn.Linear(num_features_xd*4,1024)     #表示全连接层，输入维度为78*4，输出维度为1024
        self.fc_g2 = torch.nn.Linear(1024,output_dim)            #表示全连接层，输入维度为1024，输出维度为128（潜在变量）
        self.relu = nn.ReLU()                                    #表示激活函数relu
        self.dropout = nn.Dropout(dropout)                       #表示正则化Dropout，防止过拟合

        #处理蛋白质序列的神经网络模型（1D conv）
        self.embedding_xt = nn.Embedding(num_features_xt + 1,embed_dim) #赋值嵌入层，将蛋白质序列中的离散字符映射为密集向量，num_feature_xt+1制定了输入的字符数量，embed_dim表示输出参数的维度
        self.conv_xt_1 = nn.Conv1d(in_channels=1000,out_channels=n_filters,kernel_size=8) #赋值1D卷积层，in_channels=1000指的是输入特征通道数，输出通道是32维，kernel_size表示卷积核的大小为8
        self.fc1_xt = nn.linear(32*121,output_dim)               #赋值全连接层，这里的32*121，其中32是过滤器数量n_filters，121是经过1D卷积后序列长度，序列本来在嵌入层的是128维的，这里128-8+1=121
        '''
        self.fc1_xt = nn.Linear(32*121, output_dim) 定义了一个全连接层，其中32*121是输入的大小，output_dim是输出的大小。

        在代码中，conv_xt经过了一次1D卷积操作后，得到了一个大小为[batch_size, n_filters, conv_length]的输出，这里的conv_length是卷积后的序列长度，可以根据输入序列的长度、卷积核大小以及padding方式进行计算，输出为128维。
        然后我们将这个3D的张量展平为一个2D的张量，这个展平后的大小就是n_filters*conv_length。
        最后，我们使用一个全连接层nn.Linear将其映射到输出的大小output_dim，以便进行下游的任务。

        '''

        #将分子和蛋白提取到的特征放到一起进行回归预测
        self.fc1 = nn.linear(2*output_dim,1024)      #设置全连接层，输入为2*128维，输出为1024维
        self.fc2 = nn.linear(1024,512)
        self.out = nn.linear(512,self.n_output)      #设置输出为1的回归预测

        #开始设置模型进行前向传播
    def forward(self,data):
        #SMILES图输入
        x,edge_index,batch = data.x,data.edge_index,data.batch
        #得到蛋白序列输入
        target = data.target
        '''
        data.x是SMILES图的节点特征矩阵，维度为[num_nodes, num_features_xd]，其中num_nodes是图的节点数，num_features_xd是每个节点的特征维度。
        edge_index是SMILES图的边索引矩阵，维度为[2, num_edges]，其中第一行是源节点索引，第二行是目标节点索引，表示有一条边从源节点指向目标节点。
        batch是SMILES图的batch索引，维度为[num_nodes]，用于将节点分配到不同的样本中。

        data.target是蛋白质序列的输入，维度为[batch_size, seq_len]，其中batch_size是样本数，seq_len是蛋白质序列的长度，这里是1000
        '''
        #图模型前向传播
        x = self.conv1(x,edge_index)                 #第一层GCN
        x = self.relu(x)                             #非线性激活函数
        
        x = self.conv2(x,edge_index)                 #第二层GCN
        x = self.relu(x)
 
        x = self.conv3(x,edge_index)                 #第三层GCN
        x = self.relu(x)

        x = gmp(x,batch)                             #全局最大池化,batch

        x = self.relu(self.fc_g1(x))                 #先进行全连接层，再用激活函数激活
        x = self.dropout(x)                          #正则化
        x = self.fc_g2(x)
        x = self.dropout(x)

        #蛋白序列特征提取模型
        embedded_xt = self.embedding_xt(target)      #嵌入层将离散数据连续化
        conv_xt = self.conv_xt_1(embedded_xt)        #1D卷积层

        xt = conv_xt.view(-1,32*123)                 #这里是将1D卷积提取到的特征展平转化为batch_size,32*123维也就是2维的方便下一层的全连接层输入
        xt = self.fc1_xt(xt)

        #将提取到的药物和蛋白特征放到一起
        xc = torch.cat((x,xt),1)                     #将特征按列拼接到一起
        
        #加入一些dense层
        xc = self.fc1(xc)
        xc = self.relu(xc)
        xc = self.dropout(xc)
        xc = self.f2(xc)
        xc = self.relu(xc)
        xc = self.dropout(xc)
        out = self.out(xc)
        return out


In [3]:
print(len("ABCDEFGHIKLMNOPQRSTUVWXYZ"))

25
